In [1]:
import json
import re
import orgelpredigt_analysis as oa
from rapidfuzz import fuzz
import pandas as pd

In [2]:
def fuzzy_match(a, b, threshold=80):
    similarity_score = fuzz.ratio(a, b)
    return similarity_score >= threshold

In [3]:
with open("E000052.json", "r") as f:
    sermon_chunked = json.load(f)

In [4]:
sermon = oa.Sermon("E000052")

In [5]:
known_song_quotes = list(set([q for q in sermon.all_references if q.startswith('E10')]))
with open('songs_to_pages_mapping.json') as f:
    songbook_pages = json.load(f)

relevant_pages = []
for i in known_song_quotes:
    page = songbook_pages[i]['pages']
    if page != '':
        page = str(int(page) + 42)          # add 42 to offset prefatory matter in scans
        add_page = str(int(page) + 1)
        relevant_pages.append([i, page])
        relevant_pages.append([i, add_page])
print(relevant_pages)
page_nrs = []
for p in relevant_pages:
    page_nrs.append(p[1])

[['E100057', '608'], ['E100057', '609'], ['E100134', '1039'], ['E100134', '1040'], ['E100138', '1057'], ['E100138', '1058'], ['E100132', '226'], ['E100132', '227'], ['E100017', '434'], ['E100017', '435'], ['E100022', '740'], ['E100022', '741'], ['E100067', '925'], ['E100067', '926'], ['E100078', '642'], ['E100078', '643'], ['E100141', '650'], ['E100141', '651']]


In [6]:
relevant_page_texts = []
for n in page_nrs:
    with open(f"source_texts/praxis_pietatis_verses/{n}.json") as f:
        page = json.load(f)
    page_info = {}
    page_info[n] = page
    relevant_page_texts.append(page_info)

In [7]:
relevant_page_texts

[{'608': ['der herr geschworn hat über dich',
   'und wird ihn nicht gereuen du bist ein priester ewiglich',
   'den gottsdienst zu verneuen',
   'gank nach der weiß melchisedech',
   'das alt opffer muß gar hinweg',
   'wann du dich selbst lässt tödten',
   'doch wird der herr stets bey dir seyn',
   'und stehn zu deiner rechten',
   'und ftraffen mit ewiger pein all',
   'die dir widerfechten zur zeit',
   'wann eins sein zorn ergrimmt',
   'die gwaltig könige hinnimmt',
   'wird fie mit macht zuschmeissen',
   'er wird herrlich mit grossem pracht unter den heyden richten',
   'er wird thun gar viel grosse schlacht widr die',
   'so ihn vernichten er wird dem fürsten dies fer welt',
   'der ihm sich stets zuwider stellt',
   'sein höls lisch reich zureissen',
   'in schwachheit leiden kreutz und tod',
   'wird er hic zeitlich sterben und überwinden alle noth',
   'all gottesguter erben und aufferstehn am dritten tag',
   'daß er ewig regieren may',
   'ein king über himml und erden',

In [8]:
hits = []

for i in range(len(sermon_chunked)):                # for each paragraph
    for j in range(len(sermon_chunked[i])):         # for each sentence
        query = " ".join(sermon_chunked[i][j]["words"])
        query = re.sub(r'[/.,;:?!]', '', query)
        for page in relevant_page_texts:
            for pagenr, verses in page.items():
                for verse in verses:
                    if fuzzy_match(query, verse, 80):
                        hits.append([query, verse, str(i) +  "-" + str(j), pagenr])


In [9]:
pd.DataFrame(hits, columns=["Orgelpredigtstelle", "Liedvers", "Occurrence", "Seite Liederbuch"])

,Orgelpredigtstelle,Liedvers,Occurrence,Seite Liederbuch
0,zwingt die säyten in cythara etc,zwingt die sensen in cythara,20-18,741
1,den mund zum lobe dein,deffn unsern mund zum lobe dein,21-122,608
2,den glauben mehr stärck den verstand,den glauben mehr stärck den verstand,21-124,608
3,daß uns dein nahm werd wolbekandt,daß uns dein nam sey wol bekannt,21-125,608
4,zwingt die säyten in cithara,zwingt die sensen in cythara,21-161,741
5,und last die süsse musica gantz freudenreich e...,und lafst die süss se musica gank frendenreich...,21-162,741
6,der finger an gottes rechten hand,der fikt zu gottes rechten hand,21-548,226
7,da die engel singen die nova cantica,denn da da die engel fingen nova cantica,21-724,435
8,und die schellen klingen,und die schellen klingen,21-725,435
9,weicht ihr trauergeister denn mein freuden mei...,weicht ihr trauergeiffer denn mein freuden mei...,21-751,926


In [10]:

def flatten(xss):
    return [x for xs in xss for x in xs]

In [11]:
validation = []
for i in range(len(sermon_chunked)):                # for each paragraph
    for j in range(len(sermon_chunked[i])):         # for each sentence
        if " musikwerk" in sermon_chunked[i][j]["types"]:
            line = " ".join(sermon_chunked[i][j]["words"])
            refs = ", ".join(set(flatten(sermon_chunked[i][j]["references"])))
            validation.append([line, str(i) + "-" + str(j), refs])


In [12]:
pd.DataFrame(validation, columns=["Orgelpredigtstelle", "Occurrence", "Referenzen"])

,Orgelpredigtstelle,Occurrence,Referenzen
0,zwingt die säyten in cythara etc.,20-18,E100022
1,daß es heist: mein hertz soll dir allzeit brin...,21-45,E100141
2,deines lobs gebührlichs theil/,21-46,E100141
3,auch soll meine zunge singen/,21-47,E100141
4,täglich dein unzählich heyl.,21-48,E100141
...,...,...,...
124,und die lieben propheten all/,51-84,E100078
125,die theuren märtrer allzumahl dich loben herr ...,51-85,E100078
126,mit den seraphinen prangen/,51-87,E100134
127,und das hohe lied anfangen:,51-88,E100134


In [ ]:
def sentence_to_html(sentence: dict, par_nr: int, sentence_nr: int) -> str:
    """Takes a sentence dictionary and returns an html <p>-tag with appropriate child tags
        Args:
            sentence: The dict containing the keys "words" (list), "types" (list) and "references" (list of lists)
            par_nr: The number of the paragraph
            sentence_nr: The number of the sentence in the paragraph
        Returns:
            A string containing the tag
    """

    def add_tooltip(id: str) -> str:
        tooltip = f'<span class="tooltiptext">{id}</span>'
        return tooltip

    tag = f'<span class="orgelpredigt_span" id="{par_nr}-{sentence_nr}">'

    current_tag = []

    words = sentence["words"]
    types = sentence["types"]
    refs = sentence["references"]

    if types[0] != "":
        current_tag.append(types[0].strip())
        tag += f'<span class="{types[0].strip()} tooltip">{add_tooltip(refs[0][0])}'

    for word, type, ref in zip(words, types, refs):
        if len(ref) > 0:
            thisref = ref[-1]
        else:
            thisref = ''

        if type == "":
            if len(current_tag) == 0:
                tag += f' {word}'
            else:
                tag += f'</span> {word}'
                current_tag.pop()
        elif len(current_tag) > 0:
            if type.strip() == current_tag[-1]:
                tag += f' {word}'
            else:
                tag += f'<span class="{type.strip()} tooltip">{add_tooltip(thisref)}{word}'
                current_tag.append(type.strip())
        else:
            tag += f'<span class="{type.strip()}">{add_tooltip(thisref)}{word}'
            current_tag.append(type.strip())
    
    if len(current_tag) != 0:
        tag += '</span>'

    return f'{tag}</span>'


In [21]:
with open("E000052.json", "r") as f:
    sermon_chunked = json.load(f)

sermon_html = f'<div class="orgelpredigt">'

for i, par in enumerate(sermon_chunked):
    paragraph_text = f'<p id="{i}">'
    for j, sent in enumerate(par):
        paragraph_text += sentence_to_html(sent, i, j)
    paragraph_text += "</p>"
    sermon_html += paragraph_text

sermon_html += "</div>"
print(f'<html><body>{sermon_html}</body></html>')


<html><body><div class="orgelpredigt"><p id="0"><span class="orgelpredigt_span" id="0-0"> ein wolgerührtes orgelwerck/ als eine anreitzung zur frucht des geistes/</span><span class="orgelpredigt_span" id="0-1"> hat am xiv. sonntage nach trinitatis anno 1721.</span><span class="orgelpredigt_span" id="0-2"> aus der sonst gewöhnlichen vesper lection an die<span class="bibel"><span class="tooltiptext">Tooltip text</span>galater cap.</span></span><span class="orgelpredigt_span" id="0-3"><span class="bibel tooltip"><span class="tooltiptext">Tooltip text</span> v. v. 16. biß zu ende</span> /</span><span class="orgelpredigt_span" id="0-4"> bey der einweyhung der fürtrefflichen neuen orgel/</span><span class="orgelpredigt_span" id="0-5"> in der kneiphöfischen thumkirchen/</span><span class="orgelpredigt_span" id="0-6"> in öffentlicher predigt vorgestellet/</span><span class="orgelpredigt_span" id="0-7"> und auff verlangen vieler frommer zuhörer dem druck übergeben m.</span><span class="orgelpre

In [25]:
example = sermon_chunked[39][21]

sentence_to_html(example, 39, 22)

'<span class="orgelpredigt_span" id="39-22"><span class="bibel tooltip"><span class="tooltiptext">Ex_28-35</span> 35.</span> bedencket,<span class="musikwerk"><span class="tooltiptext">[\'E100017\']</span>ubi sunt gaudia!</span></span>'